In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('dataset1') 

# Actions that we try to detect
actions = np.array(['Good Morning', 'Good Afternoon', 'Good Night'])

# Thirty videos worth of data
no_sequences = 120

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [9]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
label_map

{'Good Morning': 0, 'Good Afternoon': 1, 'Good Night': 2}

In [13]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
np.array(sequences).shape

(90, 30, 1662)

In [15]:
np.array(labels).shape

(90,)

In [16]:
X = np.array(sequences)

In [17]:
X.shape

(90, 30, 1662)

In [18]:
y = to_categorical(labels).astype(int)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [20]:
y_test.shape

(5, 3)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [9]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [10]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [24]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 7s 1s/step - loss: 1.1518 - categorical_accuracy: 0.2941
Epoch 2/2000
3/3 [==============================] - 0s 82ms/step - loss: 17.6627 - categorical_accuracy: 0.3294
Epoch 3/2000
3/3 [==============================] - 0s 53ms/step - loss: 9.4769 - categorical_accuracy: 0.3412
Epoch 4/2000
3/3 [==============================] - 0s 72ms/step - loss: 8.2398 - categorical_accuracy: 0.3412
Epoch 5/2000
3/3 [==============================] - 0s 61ms/step - loss: 3.0146 - categorical_accuracy: 0.1529
Epoch 6/2000
3/3 [==============================] - 0s 53ms/step - loss: 1.4552 - categorical_accuracy: 0.3882
Epoch 7/2000
3/3 [==============================] - 0s 94ms/step - loss: 1.7433 - categorical_accuracy: 0.4941
Epoch 8/2000
3/3 [==============================] - 0s 96ms/step - loss: 1.0989 - categorical_accuracy: 0.4471
Epoch 9/2000
3/3 [==============================] - 0s 60ms/step - loss: 4.9637 - categorical_accuracy: 0.4118
Ep

3/3 [==============================] - 0s 61ms/step - loss: 0.3988 - categorical_accuracy: 0.7176
Epoch 145/2000
3/3 [==============================] - 0s 128ms/step - loss: 0.3380 - categorical_accuracy: 0.8941
Epoch 146/2000
3/3 [==============================] - 0s 73ms/step - loss: 0.3408 - categorical_accuracy: 0.8471
Epoch 147/2000
3/3 [==============================] - 0s 76ms/step - loss: 0.3029 - categorical_accuracy: 0.9059
Epoch 148/2000
3/3 [==============================] - 0s 89ms/step - loss: 0.3060 - categorical_accuracy: 0.8353
Epoch 149/2000
3/3 [==============================] - 0s 105ms/step - loss: 0.3551 - categorical_accuracy: 0.9059
Epoch 150/2000
3/3 [==============================] - 0s 73ms/step - loss: 0.3108 - categorical_accuracy: 0.8941
Epoch 151/2000
3/3 [==============================] - 0s 76ms/step - loss: 0.3061 - categorical_accuracy: 0.8706
Epoch 152/2000
3/3 [==============================] - 0s 120ms/step - loss: 0.2834 - categorical_accuracy: 0.

3/3 [==============================] - 0s 118ms/step - loss: 0.1503 - categorical_accuracy: 0.9647
Epoch 287/2000
3/3 [==============================] - 0s 95ms/step - loss: 0.1368 - categorical_accuracy: 0.9529
Epoch 288/2000
3/3 [==============================] - 0s 78ms/step - loss: 0.1256 - categorical_accuracy: 0.9647
Epoch 289/2000
3/3 [==============================] - 0s 73ms/step - loss: 0.0927 - categorical_accuracy: 0.9647
Epoch 290/2000
3/3 [==============================] - 0s 69ms/step - loss: 0.0743 - categorical_accuracy: 0.9882
Epoch 291/2000
3/3 [==============================] - 0s 66ms/step - loss: 0.0637 - categorical_accuracy: 1.0000
Epoch 292/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.0691 - categorical_accuracy: 0.9765
Epoch 293/2000
3/3 [==============================] - 1s 200ms/step - loss: 0.0559 - categorical_accuracy: 1.0000
Epoch 294/2000
3/3 [==============================] - 0s 92ms/step - loss: 0.0393 - categorical_accuracy: 1.0

3/3 [==============================] - 1s 390ms/step - loss: 2.9109e-04 - categorical_accuracy: 1.0000
Epoch 426/2000
3/3 [==============================] - 0s 78ms/step - loss: 2.8701e-04 - categorical_accuracy: 1.0000
Epoch 427/2000
3/3 [==============================] - 0s 99ms/step - loss: 2.8752e-04 - categorical_accuracy: 1.0000
Epoch 428/2000
3/3 [==============================] - 0s 100ms/step - loss: 2.8445e-04 - categorical_accuracy: 1.0000
Epoch 429/2000
3/3 [==============================] - 1s 329ms/step - loss: 2.7786e-04 - categorical_accuracy: 1.0000
Epoch 430/2000
3/3 [==============================] - 0s 73ms/step - loss: 2.7374e-04 - categorical_accuracy: 1.0000
Epoch 431/2000
3/3 [==============================] - 0s 112ms/step - loss: 2.7215e-04 - categorical_accuracy: 1.0000
Epoch 432/2000
3/3 [==============================] - 0s 83ms/step - loss: 2.6988e-04 - categorical_accuracy: 1.0000
Epoch 433/2000
3/3 [==============================] - 1s 414ms/step - loss:

Epoch 563/2000
3/3 [==============================] - 1s 324ms/step - loss: 9.9909e-05 - categorical_accuracy: 1.0000
Epoch 564/2000
3/3 [==============================] - 1s 214ms/step - loss: 9.8976e-05 - categorical_accuracy: 1.0000
Epoch 565/2000
3/3 [==============================] - 1s 272ms/step - loss: 9.7678e-05 - categorical_accuracy: 1.0000
Epoch 566/2000
3/3 [==============================] - 1s 340ms/step - loss: 9.7178e-05 - categorical_accuracy: 1.0000
Epoch 567/2000
3/3 [==============================] - 1s 275ms/step - loss: 9.6619e-05 - categorical_accuracy: 1.0000
Epoch 568/2000
3/3 [==============================] - 0s 200ms/step - loss: 9.6313e-05 - categorical_accuracy: 1.0000
Epoch 569/2000
3/3 [==============================] - 1s 237ms/step - loss: 9.4979e-05 - categorical_accuracy: 1.0000
Epoch 570/2000
3/3 [==============================] - 1s 215ms/step - loss: 9.4433e-05 - categorical_accuracy: 1.0000
Epoch 571/2000
3/3 [==============================] - 0s

...

3/3 [==============================] - 0s 64ms/step - loss: 4.7262e-06 - categorical_accuracy: 1.0000
Epoch 1528/2000
3/3 [==============================] - 0s 130ms/step - loss: 4.7234e-06 - categorical_accuracy: 1.0000
Epoch 1529/2000
3/3 [==============================] - 1s 188ms/step - loss: 4.7052e-06 - categorical_accuracy: 1.00000s - loss: 4.7052e-06 - categorical_accuracy: 1.00
Epoch 1530/2000
3/3 [==============================] - 0s 90ms/step - loss: 4.6996e-06 - categorical_accuracy: 1.0000
Epoch 1531/2000
3/3 [==============================] - 0s 82ms/step - loss: 4.6856e-06 - categorical_accuracy: 1.0000
Epoch 1532/2000
3/3 [==============================] - 0s 65ms/step - loss: 4.6786e-06 - categorical_accuracy: 1.0000
Epoch 1533/2000
3/3 [==============================] - 0s 70ms/step - loss: 4.6701e-06 - categorical_accuracy: 1.0000
Epoch 1534/2000
3/3 [==============================] - 0s 58ms/step - loss: 4.6547e-06 - categorical_accuracy: 1.0000
Epoch 1535/2000
3/3 

Epoch 1596/2000
3/3 [==============================] - 0s 167ms/step - loss: 4.0643e-06 - categorical_accuracy: 1.0000
Epoch 1597/2000
3/3 [==============================] - 0s 103ms/step - loss: 4.0559e-06 - categorical_accuracy: 1.0000
Epoch 1598/2000
3/3 [==============================] - 0s 100ms/step - loss: 4.0503e-06 - categorical_accuracy: 1.0000
Epoch 1599/2000
3/3 [==============================] - 0s 52ms/step - loss: 4.0419e-06 - categorical_accuracy: 1.0000
Epoch 1600/2000
3/3 [==============================] - 0s 61ms/step - loss: 4.0250e-06 - categorical_accuracy: 1.0000
Epoch 1601/2000
3/3 [==============================] - 0s 57ms/step - loss: 4.0152e-06 - categorical_accuracy: 1.0000
Epoch 1602/2000
3/3 [==============================] - 0s 74ms/step - loss: 4.0236e-06 - categorical_accuracy: 1.0000
Epoch 1603/2000
3/3 [==============================] - 0s 111ms/step - loss: 4.0054e-06 - categorical_accuracy: 1.0000
Epoch 1604/2000
3/3 [==============================]

3/3 [==============================] - 0s 117ms/step - loss: 3.4963e-06 - categorical_accuracy: 1.0000
Epoch 1666/2000
3/3 [==============================] - 0s 149ms/step - loss: 3.4879e-06 - categorical_accuracy: 1.0000
Epoch 1667/2000
3/3 [==============================] - 0s 160ms/step - loss: 3.4837e-06 - categorical_accuracy: 1.0000
Epoch 1668/2000
3/3 [==============================] - 0s 175ms/step - loss: 3.4767e-06 - categorical_accuracy: 1.0000
Epoch 1669/2000
3/3 [==============================] - 1s 222ms/step - loss: 3.4654e-06 - categorical_accuracy: 1.0000
Epoch 1670/2000
3/3 [==============================] - 1s 163ms/step - loss: 3.4584e-06 - categorical_accuracy: 1.0000
Epoch 1671/2000
3/3 [==============================] - 1s 182ms/step - loss: 3.4556e-06 - categorical_accuracy: 1.0000
Epoch 1672/2000
3/3 [==============================] - 1s 240ms/step - loss: 3.4430e-06 - categorical_accuracy: 1.0000
Epoch 1673/2000
3/3 [==============================] - 0s 93ms/s

3/3 [==============================] - 0s 121ms/step - loss: 2.6044e-06 - categorical_accuracy: 1.0000
Epoch 1802/2000
3/3 [==============================] - 0s 110ms/step - loss: 2.6001e-06 - categorical_accuracy: 1.0000
Epoch 1803/2000
3/3 [==============================] - 1s 341ms/step - loss: 2.5931e-06 - categorical_accuracy: 1.0000
Epoch 1804/2000
3/3 [==============================] - 0s 67ms/step - loss: 2.5903e-06 - categorical_accuracy: 1.0000
Epoch 1805/2000
3/3 [==============================] - 0s 82ms/step - loss: 2.5847e-06 - categorical_accuracy: 1.0000
Epoch 1806/2000
3/3 [==============================] - 0s 76ms/step - loss: 2.5805e-06 - categorical_accuracy: 1.0000
Epoch 1807/2000
3/3 [==============================] - 0s 92ms/step - loss: 2.5777e-06 - categorical_accuracy: 1.0000
Epoch 1808/2000
3/3 [==============================] - 0s 84ms/step - loss: 2.5679e-06 - categorical_accuracy: 1.0000 0s - loss: 2.6803e-06 - categorical_accuracy: 1.
Epoch 1809/2000
3/3 

3/3 [==============================] - 0s 62ms/step - loss: 1.9578e-06 - categorical_accuracy: 1.0000
Epoch 1939/2000
3/3 [==============================] - 0s 100ms/step - loss: 1.9606e-06 - categorical_accuracy: 1.0000
Epoch 1940/2000
3/3 [==============================] - 1s 340ms/step - loss: 1.9564e-06 - categorical_accuracy: 1.0000
Epoch 1941/2000
3/3 [==============================] - 0s 140ms/step - loss: 1.9522e-06 - categorical_accuracy: 1.0000
Epoch 1942/2000
3/3 [==============================] - 0s 125ms/step - loss: 1.9466e-06 - categorical_accuracy: 1.0000
Epoch 1943/2000
3/3 [==============================] - 0s 178ms/step - loss: 1.9438e-06 - categorical_accuracy: 1.0000
Epoch 1944/2000
3/3 [==============================] - 1s 215ms/step - loss: 1.9382e-06 - categorical_accuracy: 1.0000
Epoch 1945/2000
3/3 [==============================] - 0s 119ms/step - loss: 1.9340e-06 - categorical_accuracy: 1.0000
Epoch 1946/2000
3/3 [==============================] - 0s 147ms/s

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

In [27]:
res = model.predict(X_test)

In [28]:
actions[np.argmax(res[4])]

'Good Night'

In [29]:
actions[np.argmax(y_test[4])]

'Good Night'

In [30]:
model.save('mod1.h5')

In [11]:
model.load_weights('mod1.h5')

In [32]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [33]:

yhat = model.predict(X_test)

In [34]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [35]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 1]],

       [[4, 0],
        [0, 1]],

       [[2, 0],
        [0, 3]]], dtype=int64)

In [36]:
accuracy_score(ytrue, yhat)

1.0

In [12]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.py

Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_bas

Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 

Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_bas

Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Morning
<class 'mediapipe.python.solution

Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good Night
<class 'mediapipe.python.solution_bas